Kyler Gappa

River Kelly

CSCI-347 Data Mining

Final Project

<https://archive.ics.uci.edu/ml/datasets/Liver+Disorders>

# Setup Code

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt